In [ ]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

# Multiple Choice Mode

In [ ]:
import json

# 读取 JSON 文件
with open("mc_task.json", "r", encoding="utf-8") as f:
    data = json.load(f)

tests = []

for item in data:
    question = item["question"]
    options = list(item["mc1_targets"].keys())
    # 找到正确答案（值为 1 的那个选项）
    correct_answer = [k for k, v in item["mc1_targets"].items() if v == 1][0]

    # 构造选项文本（动态字母序列）
    options_text = "\n".join([f"{chr(65+i)}: {opt}" for i, opt in enumerate(options)])

    # 拼接最终格式
    test_str = f"""<|im_start|>user
Question: {question}
{options_text}
Which one is correct?
<|im_end|>\n<|im_start|>assistant\n"""

    tests.append(test_str)

# 示例打印前 1 个
print(tests[0])


In [ ]:
example_answers = llm.generate(
    tests,
    SamplingParams(
        temperature=0,
        max_tokens=1024,
        skip_special_tokens=False,
    ),
)

In [ ]:
answers = [ans.outputs[0].text for ans in example_answers]

# 存成 JSON 文件
with open("answers.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=2)

# 打印前 20 个
for ans in loaded_answers[:20]:
    print(ans)

# QA Mode

In [ ]:
import json

# 读取 JSON 文件
with open("mc_task.json", "r", encoding="utf-8") as f:
    data = json.load(f)

tests = []

for item in data:
    question = item["question"]
    test_str = f"""<|im_start|>user
Given a question, generate a 1-2 sentence answer.
Question: {question}
<|im_end|>\n<|im_start|>assistant\n"""

    tests.append(test_str)

In [ ]:
example_answers = llm.generate(
    tests,
    SamplingParams(
        temperature=0,
        max_tokens=256,
        skip_special_tokens=False,
    ),
)

In [ ]:
answers = [ans.outputs[0].text for ans in example_answers]

# 存成 JSON 文件
with open("qa_answers.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=2)